In [ ]:
# --- Необходимые функции для присвоения займам score-рейтинг ---



# ---
# Функция поиска актуального скоринга заёмщика
def act_score_wo_ee(user_id, date, letter=True):
    
    metascore = score[score.user_id == user_id] 
    metascore.loc[~metascore['date_of_report'].isna(), 'created'] = metascore.loc[:,'date_of_report'].apply(lambda x: x - timedelta(hours=3))
    
    metascore = metascore[(metascore.created <= date)
                          & (metascore.created >= (date - timedelta(days=30)))] \
                          .sort_values("created", ascending=False) 
    
    Score = 0
    if len(metascore) > 0:
        for i in range(metascore.id.count()):
             if metascore.iloc[i, 7] > 0:
                    Score = metascore.iloc[i, 7]
                    break 
                    
        if Score == 0: 
            Score = np.nan 

    else: 
        Score = np.nan 

    if letter: 
        return(get_letter_rating(Score)) 
    else:
        return(Score)  



# ---
# Функция перевода score в буквенный рейтинг
def get_letter_rating(df): 
    if df >= 799: 
        rating = 'A'
    elif (df >= 756) and (df <= 798): 
        rating = 'B'
    elif (df >= 709) and (df <= 755): 
        rating = 'C'
    elif (df >= 649) and (df <= 708): 
        rating = 'D'
    elif (df >= 572) and (df <= 648): 
        rating = 'E'
    elif (df >= 1) and (df <= 571): 
        rating = 'F'
    else: 
        rating = 'Nr'
        
    return rating



# ---
# Присваиваем рейтинг позьзователя на момент создания заявки на займ
loans_p2p_report['rating_letter'] = loans_p2p_report.apply(lambda x: act_score_wo_ee(x['owner_id'], x['date_created'], letter=True), axis=1)